# 태스크 6: LangGraph와 Bedrock 모델 통합

이 노트북에서는 에이전트가 사용할 수 있는 도구를 사용하여 작업 순서를 결정하고, 이를 구현하는 계획 및 실행 에이전트를 사용하는 방법을 알아보겠습니다. 

특정 애플리케이션은 사용자의 질문에 답변하기 위해 언어 모델과 다양한 유틸리티에 대한 적응 가능한 호출 순서를 요구합니다. LangChain 에이전트 인터페이스는 유연하며, 외부 도구와 LLM의 추론을 통합할 수 있습니다. 에이전트는 사용자 입력에 따라 사용할 도구를 선택할 수 있습니다. 에이전트는 여러 도구를 사용할 수 있으며 한 도구의 출력을 다음 도구의 입력으로 활용할 수 있습니다.

## 태스크 6.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
!pip show langchain langchain-core langgraph

In [ ]:
from typing import Annotated
import pandas as pd
import numexpr

# LangChain / LangGraph imports
from langchain.tools import tool
from langchain_aws import ChatBedrock
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.graph import StateGraph, END

In [ ]:
# ----------------------------------------
# 🧰 Tools 정의
# ----------------------------------------

@tool
def lookup_price(item_name: str) -> str:
    """Lookup the price of a product by its name from the CSV file."""
    df = pd.read_csv("sales.csv")
    result = df[df["product_id"] == item_name]
    if result.empty:
        return f"'{item_name}' not found."
    return str(result["price"].values[0])


@tool
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression and return the result as a string."""
    try:
        result = numexpr.evaluate(expression)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"


@tool
def echo_tool(text: str) -> str:
    """Echo back the provided text."""
    return f"Echo: {text}"

In [ ]:
# ----------------------------------------
# 🧩 Agent Node 생성
# ----------------------------------------

# Bedrock 모델 (또는 호환 모델)
llm = ChatBedrock(
    model_id="amazon.nova-lite-v1:0",
    region="us-east-1"
)

In [ ]:
# 사용할 도구 리스트
tools = [lookup_price, calculator, echo_tool]

# create_react_agent: LLM이 ToolNode를 통해 도구를 호출하도록 자동 구성
agent = create_react_agent(llm, tools)

In [ ]:
print("=== LangGraph ReAct Agent Example ===\n")
user_input = "사용자 질문:How much will it cost to buy 3 units of P002 and 5 units of P003?"

# ✅ agent 자체가 실행 가능한 Graph입니다
result = agent.invoke({"messages": [HumanMessage(content=user_input)]})

print("\n🤖 Agent Response:")
print(result["messages"][-1].content)